In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('KC_case_data.csv', parse_dates = ['date'])
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [3]:
df.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

MAU февраля

In [4]:
df['month'] = df.date.dt.month_name()
df = df.reindex \
(columns = ['month',\
            'date', \
            'event', \
            'purchase_sum', \
            'os_name', \
            'device_id', \
            'gender', \
            'city', \
            'utm_source'])
febmau = df.groupby(['month'], as_index = False).agg({'device_id': 'nunique'})
febmau

,month,device_id
0,February,75032
1,January,99161
2,March,74623


Кол-во установок в январе

In [5]:
janinstall = df.query('month == "January" & event == "app_install"')
janinstall.shape[0]

80297

Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? Ответ впишите в формате: дд.мм.гггг

Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [6]:
df

,month,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,January,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,January,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,January,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,January,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,January,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...,...
2747963,March,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,March,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,March,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,March,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


In [7]:
# Отберем только нужные столбцы
data_event = df[['date','event','device_id']]

# Здесь только установки
event_installs = data_event.query('event == "app_install"')

# Здесь покупки все
event_purchase = data_event.query('event == "purchase"')

# Первые покупки каждого
event_purchase_first = event_purchase.set_index('device_id').groupby(level=0)['date'].min().reset_index()

# Обьеденим установки и  ПЕРВЫЕ покупки
data_event_first = event_installs.merge(event_purchase_first, how = 'left', on = 'device_id')

# создадим столбец с количеством дней между установкой и 1 покупкой ПЕРВОЙ !!!
data_event_first['days'] = (data_event_first['date_y'] - data_event_first['date_x'])/ np.timedelta64 ( 1 , 'D')

# Отберем только где покупка произошла не более чем через 7 дней ПЕРВЫЕ ПОКУПКИ
data_event_first = data_event_first.query('days < 8')[['device_id', 'date_y', 'days']]

# СНОВА обьеденим установки и покупки которые произошли в течении 7 дней ПЕРВЫЕ ПОКУПКИ

data_event_first = event_installs.merge(data_event_first, how = 'left', on = 'device_id')

# Групируем !! ОПЯТЬ ПЕРВЫЕ ПОКУПКИ
data_event_first = data_event_first \
                                    .groupby('date') \
                                    .agg({'device_id': 'count', 'days': 'count'}) \
                                    .rename(columns = {'days':'purchase',
                                                       'device_id':'app_install'})
# считаем конверсию в покупку ПЕРВУЮ ПОКУПКУ !!!(создаем столбец)

data_event_first['CR7days'] = data_event_first.purchase/data_event_first.app_install

#Находим дату с максимальным CR
data_event_first.CR7days.idxmax()

Timestamp('2020-01-01 00:00:00')

Укажите значение CR из предыдущего вопроса в % (округлите до 1 цифры после запятой). Формат числа - с точкой. Пример: 66.4.

In [8]:
round(data_event_first.loc[data_event_first.CR7days.idxmax()].CR7days * 100, 1)

39.3

С какого платного маркетингового канала пришло больше всего новых пользователей? 

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

In [10]:
new_users_per_channel = df \
                            .query('event == "app_install"') \
                            .groupby(['utm_source'], as_index = False) \
                            .agg({'device_id': 'count'}) \
                            .rename(columns = {'device_id':'quantity'}) \
                            .sort_values('quantity', ascending = False)
new_users_per_channel

,utm_source,quantity
0,-,32460
6,yandex-direct,29368
2,google_ads,26286
5,vk_ads,23189
3,instagram_ads,20096
1,facebook_ads,13916
4,referal,9282


Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате: «Поиск», «Добавление товара», «Переход в корзину», «Регистрация».

Подсказка: 

- Для того, чтобы выделить группу тех, кому нужно и не нужно регистрироваться, добавьте колонку с датой регистрации. Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован. 

In [46]:
# Дата первой регистрации:
first_reg = df.query('event == "register"').sort_values('date').drop_duplicates('device_id')

# Добавляем колонку с датой:
df['first_reg'] = df['device_id'].map(first_reg.set_index('device_id') ['date'])

# Те, кто уже был зарегестрирован
already_reg = df[df['first_reg'] < df['date']]

# Разбивка по каждому действию
already_reg.groupby('event')['device_id'].nunique()

#Считаем конверсию:

print('Конверсия в поиск из открытия:' \
      , round((40482 / 40991) * 100, 2) \
      , 'Конверсия в выбор товара из поиска:' \
      , round((37926 / 40482) * 100, 2) \
      , 'Конверсия в перерход в корзину после выбора:' \
      , round((34517 / 37926) * 100, 2) \
      , 'Конверсия в покупку из корзины:' \
      , round((24880 / 34517) * 100, 2), sep = '\n')

Конверсия в поиск из открытия:
98.76
Конверсия в выбор товара из поиска:
93.69
Конверсия в перерход в корзину после выбора:
91.01
Конверсия в покупку из корзины:
72.08


In [29]:
df

,month,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,first_reg
0,January,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-07
1,January,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,January,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,January,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,January,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,NaT
...,...,...,...,...,...,...,...,...,...,...
2747963,March,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-31
2747964,March,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31
2747965,March,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31
2747966,March,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-31
